In [5]:
import csv
from collections import defaultdict

class DashboardPuro:
    def __init__(self):
        self.csvs = {}
        print("✅ Dashboard iniciado (Python puro)")
        print("💡 Usa: dashboard.ayuda() para ver comandos")
    
    def cargar(self, nombre, ruta):
        """Cargar un archivo CSV"""
        try:
            with open(ruta, 'r', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                datos = list(reader)
                self.csvs[nombre] = datos
                print(f"✅ {nombre} cargado: {len(datos)} filas")
                if datos:
                    print(f"   Columnas: {list(datos[0].keys())}")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def ver(self, nombre, n=10):
        """Ver primeras n filas de un CSV"""
        if nombre not in self.csvs:
            print(f"❌ {nombre} no está cargado")
            return
        
        datos = self.csvs[nombre][:n]
        if not datos:
            print("Sin datos")
            return
        
        # Obtener columnas
        columnas = list(datos[0].keys())
        
        # Calcular anchos
        anchos = {col: len(col) for col in columnas}
        for fila in datos:
            for col in columnas:
                anchos[col] = max(anchos[col], len(str(fila[col])))
        
        # Imprimir cabecera
        print("\n" + "="*80)
        cabecera = " | ".join([col.ljust(anchos[col]) for col in columnas])
        print(cabecera)
        print("-"*80)
        
        # Imprimir datos
        for fila in datos:
            linea = " | ".join([str(fila[col]).ljust(anchos[col]) for col in columnas])
            print(linea)
        print("="*80)
    
    def listar(self):
        """Listar archivos cargados"""
        if not self.csvs:
            print("📂 No hay archivos cargados")
        else:
            print(f"📂 Archivos cargados: {', '.join(self.csvs.keys())}")
    
    # ============ REPORTES ============
    
    def ranking_clientes(self, top=10):
        """🏆 Ranking de Clientes por Total"""
        if "clientes" not in self.csvs or "facturas_enc" not in self.csvs:
            print("⚠️ Carga clientes y facturas_enc")
            return
        
        clientes = self.csvs["clientes"]
        facturas = self.csvs["facturas_enc"]
        
        # Crear diccionario id_cliente -> nombre
        id_a_nombre = {c["id_cliente"]: c["nombre"] for c in clientes}
        
        # Sumar totales por cliente
        totales = defaultdict(float)
        for f in facturas:
            id_cli = f["id_cliente"]
            if id_cli in id_a_nombre:
                totales[id_a_nombre[id_cli]] += float(f["total"])
        
        # Ordenar y tomar top
        ranking = sorted(totales.items(), key=lambda x: x[1], reverse=True)[:top]
        
        self._imprimir_reporte("🏆 Ranking de Clientes", ranking, "Cliente", "Total")
        return ranking
    
    def ticket_promedio(self, top=10):
        """💰 Ticket Promedio por Cliente"""
        if "clientes" not in self.csvs or "facturas_enc" not in self.csvs:
            print("⚠️ Carga clientes y facturas_enc")
            return
        
        clientes = self.csvs["clientes"]
        facturas = self.csvs["facturas_enc"]
        
        # Crear diccionario id_cliente -> nombre
        id_a_nombre = {c["id_cliente"]: c["nombre"] for c in clientes}
        
        # Sumar totales y contar facturas por cliente
        totales = defaultdict(float)
        conteos = defaultdict(int)
        
        for f in facturas:
            id_cli = f["id_cliente"]
            if id_cli in id_a_nombre:
                nombre = id_a_nombre[id_cli]
                totales[nombre] += float(f["total"])
                conteos[nombre] += 1
        
        # Calcular promedios
        promedios = {nombre: totales[nombre]/conteos[nombre] for nombre in totales}
        
        # Ordenar y tomar top
        ranking = sorted(promedios.items(), key=lambda x: x[1], reverse=True)[:top]
        
        self._imprimir_reporte("💰 Ticket Promedio", ranking, "Cliente", "Promedio")
        return ranking
    
    def facturas_altas(self, top=10):
        """📈 Facturas más Altas"""
        if "facturas_enc" not in self.csvs:
            print("⚠️ Carga facturas_enc")
            return
        
        facturas = self.csvs["facturas_enc"]
        
        # Ordenar por total
        ordenadas = sorted(facturas, key=lambda x: float(x["total"]), reverse=True)[:top]
        
        resultado = [(f["id_factura"], float(f["total"])) for f in ordenadas]
        
        self._imprimir_reporte("📈 Facturas Más Altas", resultado, "ID Factura", "Total")
        return resultado
    
    def ventas_por_mes(self):
        """📅 Ventas por Mes"""
        if "facturas_enc" not in self.csvs:
            print("⚠️ Carga facturas_enc")
            return
        
        facturas = self.csvs["facturas_enc"]
        
        # Agrupar por mes (formato YYYY-MM)
        ventas = defaultdict(float)
        for f in facturas:
            try:
                fecha = f["fecha"]
                mes = fecha[:7]  # Tomar YYYY-MM
                ventas[mes] += float(f["total"])
            except:
                continue
        
        # Ordenar por mes
        resultado = sorted(ventas.items())
        
        self._imprimir_reporte("📅 Ventas por Mes", resultado, "Mes", "Total")
        return resultado
    
    def producto_mas_vendido(self, top=5):
        """🎯 Productos Más Vendidos"""
        if "facturas_det" not in self.csvs or "productos" not in self.csvs:
            print("⚠️ Carga facturas_det y productos")
            return
        
        fdet = self.csvs["facturas_det"]
        productos = self.csvs["productos"]
        
        # Diccionario id_producto -> nombre
        id_a_nombre = {p["id_producto"]: p["nombre"] for p in productos}
        
        # Sumar cantidades por producto
        cantidades = defaultdict(int)
        for f in fdet:
            id_prod = f["id_producto"]
            if id_prod in id_a_nombre:
                cantidades[id_a_nombre[id_prod]] += int(f["cantidad"])
        
        # Ordenar y tomar top
        ranking = sorted(cantidades.items(), key=lambda x: x[1], reverse=True)[:top]
        
        self._imprimir_reporte("🎯 Productos Más Vendidos", ranking, "Producto", "Cantidad")
        return ranking
    
    def ventas_por_rubro(self):
        """📦 Ventas por Rubro"""
        if "facturas_det" not in self.csvs or "productos" not in self.csvs or "rubros" not in self.csvs:
            print("⚠️ Carga facturas_det, productos y rubros")
            return
        
        fdet = self.csvs["facturas_det"]
        productos = self.csvs["productos"]
        rubros = self.csvs["rubros"]
        
        # Diccionarios de mapeo
        id_prod_a_rubro = {p["id_producto"]: p["id_rubro"] for p in productos}
        id_rubro_a_nombre = {r["id_rubro"]: r["nombre"] for r in rubros}
        
        # Sumar cantidades por rubro
        cantidades = defaultdict(int)
        for f in fdet:
            id_prod = f["id_producto"]
            if id_prod in id_prod_a_rubro:
                id_rub = id_prod_a_rubro[id_prod]
                if id_rub in id_rubro_a_nombre:
                    cantidades[id_rubro_a_nombre[id_rub]] += int(f["cantidad"])
        
        # Ordenar
        resultado = sorted(cantidades.items(), key=lambda x: x[1], reverse=True)
        
        self._imprimir_reporte("📦 Ventas por Rubro", resultado, "Rubro", "Cantidad")
        return resultado
    
    def top_productos_facturacion(self, top=10):
        """💵 Top Productos por Facturación"""
        if "facturas_det" not in self.csvs or "productos" not in self.csvs:
            print("⚠️ Carga facturas_det y productos")
            return
        
        fdet = self.csvs["facturas_det"]
        productos = self.csvs["productos"]
        
        # Diccionarios
        id_a_nombre = {p["id_producto"]: p["nombre"] for p in productos}
        id_a_precio = {p["id_producto"]: float(p["precio_unitario"]) for p in productos}
        
        # Calcular importes
        importes = defaultdict(float)
        for f in fdet:
            id_prod = f["id_producto"]
            if id_prod in id_a_nombre:
                cantidad = int(f["cantidad"])
                precio = id_a_precio[id_prod]
                importes[id_a_nombre[id_prod]] += cantidad * precio
        
        # Ordenar y tomar top
        ranking = sorted(importes.items(), key=lambda x: x[1], reverse=True)[:top]
        
        self._imprimir_reporte("💵 Top Productos por Facturación", ranking, "Producto", "Importe")
        return ranking
    
    def top_ventas(self, top=10):
        """🛒 Top Ventas (detecta columnas automáticamente)"""
        if "ventas" not in self.csvs:
            print("⚠️ Carga el archivo ventas")
            return
        
        ventas = self.csvs["ventas"]
        if not ventas:
            print("⚠️ Sin datos")
            return
        
        # Detectar columnas
        columnas = list(ventas[0].keys())
        
        # Buscar primera columna categórica (texto)
        col_cat = None
        col_num = None
        
        for col in columnas:
            try:
                float(ventas[0][col])
                if col_num is None:
                    col_num = col
            except:
                if col_cat is None:
                    col_cat = col
        
        if not col_cat or not col_num:
            print("⚠️ No se detectaron columnas válidas")
            return
        
        # Agrupar y sumar
        totales = defaultdict(float)
        for v in ventas:
            totales[v[col_cat]] += float(v[col_num])
        
        # Ordenar y tomar top
        ranking = sorted(totales.items(), key=lambda x: x[1], reverse=True)[:top]
        
        self._imprimir_reporte(f"🛒 Top Ventas: {col_cat} por {col_num}", ranking, col_cat, col_num)
        return ranking
    
    def resumen(self):
        """📊 Resumen de todos los archivos cargados"""
        print("\n" + "="*80)
        print("📊 RESUMEN DE DATOS CARGADOS")
        print("="*80)
        
        for nombre, datos in self.csvs.items():
            if datos:
                columnas = list(datos[0].keys())
                print(f"\n📄 {nombre}")
                print(f"   Filas: {len(datos)}")
                print(f"   Columnas: {', '.join(columnas)}")
        
        print("\n" + "="*80)
    
    # ============ EXPORTAR ============
    
    def exportar_csv(self, datos, nombre_archivo):
        """Exportar datos a CSV"""
        if not datos:
            print("⚠️ No hay datos para exportar")
            return
        
        try:
            with open(nombre_archivo, 'w', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                # Escribir datos (tuplas)
                for fila in datos:
                    writer.writerow(fila)
            
            print(f"✅ Exportado: {nombre_archivo}")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def exportar_txt(self, datos, titulo, nombre_archivo):
        """Exportar reporte a TXT"""
        if not datos:
            print("⚠️ No hay datos para exportar")
            return
        
        try:
            with open(nombre_archivo, 'w', encoding='utf-8') as f:
                f.write(titulo + "\n")
                f.write("="*60 + "\n\n")
                
                for item in datos:
                    f.write(f"{item[0]}: {item[1]}\n")
            
            print(f"✅ Exportado: {nombre_archivo}")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    # ============ FUNCIONES INTERNAS ============
    
    def _imprimir_reporte(self, titulo, datos, col1, col2):
        """Imprimir reporte formateado"""
        print("\n" + "="*80)
        print(titulo)
        print("="*80)
        
        if not datos:
            print("Sin datos")
            return
        
        # Calcular anchos
        ancho1 = max(len(col1), max(len(str(d[0])) for d in datos))
        ancho2 = max(len(col2), 10)
        
        # Cabecera
        print(f"{col1.ljust(ancho1)} | {col2.rjust(ancho2)}")
        print("-"*80)
        
        # Datos
        for item in datos:
            print(f"{str(item[0]).ljust(ancho1)} | {item[1]:>10.2f}")
        
        print("="*80)
    
    def ayuda(self):
        """Mostrar ayuda"""
        print("""
╔════════════════════════════════════════════════════════════════╗
║           DASHBOARD CSV - PYTHON PURO (sin librerías)          ║
╚════════════════════════════════════════════════════════════════╝

📁 CARGAR DATOS:
   dashboard.cargar('clientes', 'ruta/clientes.csv')
   dashboard.cargar('facturas_enc', 'facturas_enc.csv')
   dashboard.cargar('ventas', 'ventas.csv')

👀 VER DATOS:
   dashboard.ver('clientes', 20)        # Ver 20 filas
   dashboard.listar()                    # Ver archivos cargados
   dashboard.resumen()                   # Resumen completo

📊 REPORTES:
   dashboard.ranking_clientes(10)        # Top 10 clientes
   dashboard.ticket_promedio(10)         # Ticket promedio
   dashboard.facturas_altas(10)          # Facturas más altas
   dashboard.ventas_por_mes()            # Ventas mensuales
   dashboard.producto_mas_vendido(5)     # Top 5 productos
   dashboard.ventas_por_rubro()          # Por rubro
   dashboard.top_productos_facturacion() # Top por $
   dashboard.top_ventas(10)              # Top ventas generales

💾 EXPORTAR:
   resultado = dashboard.ranking_clientes()
   dashboard.exportar_csv(resultado, 'ranking.csv')
   dashboard.exportar_txt(resultado, 'Ranking', 'ranking.txt')

💡 Todos los reportes retornan los datos para que puedas usarlos.
        """)

# ============ INICIO ============
print("╔════════════════════════════════════════════════════════════════╗")
print("║    Dashboard CSV - Python Puro (sin librerías externas)       ║")
print("╚════════════════════════════════════════════════════════════════╝")
print("\n💡 Ejecuta: dashboard = DashboardPuro()")
print("💡 Usa: dashboard.ayuda() para ver todos los comandos\n")

╔════════════════════════════════════════════════════════════════╗
║    Dashboard CSV - Python Puro (sin librerías externas)       ║
╚════════════════════════════════════════════════════════════════╝

💡 Ejecuta: dashboard = DashboardPuro()
💡 Usa: dashboard.ayuda() para ver todos los comandos



In [9]:
# Iniciar
dashboard = DashboardPuro()

# Cargar archivos
dashboard.cargar('clientes', 'clientes.csv')
dashboard.cargar('facturas_enc', 'facturas_enc.csv')
dashboard.cargar('ventas', 'ventas.csv')

# Ver datos
dashboard.ver('clientes', 10)
dashboard.resumen()

# Ejecutar reportes
dashboard.ranking_clientes(10)
dashboard.top_ventas()
dashboard.ventas_por_mes()

# Exportar
resultado = dashboard.ranking_clientes()
dashboard.exportar_csv(resultado, 'ranking.csv')
dashboard.exportar_txt(resultado, 'Ranking de Clientes', 'ranking.txt')

✅ Dashboard iniciado (Python puro)
💡 Usa: dashboard.ayuda() para ver comandos
✅ clientes cargado: 5 filas
   Columnas: ['id_cliente', 'nombre', 'apellido', 'email', 'telefono', 'direccion', 'id_localidad']
✅ facturas_enc cargado: 1 filas
   Columnas: ['id_factura', 'fecha', 'id_cliente', 'id_sucursal', 'total']
✅ ventas cargado: 1 filas
   Columnas: ['id_venta', 'id_sucursal', 'id_producto', 'fecha', 'cantidad', 'precio_venta']

id_cliente | nombre   | apellido | email                    | telefono   | direccion | id_localidad
--------------------------------------------------------------------------------
1          | maxi     | zapata   | maxizapata@gmail.com     | 1          | 115       | 2           
2          | mariano  | gordillo | marianoelnegro@gmail.com | 13333      | calle 13  | 2           
3          | nicolas  | martinez | nicoellol@gmail.com      | 22211      | 12        | 2           
6          | Gabriela | Toons    | gabztoons@gmail.com      | 11111      | aa 11     |

In [10]:
dashboard.ayuda()


╔════════════════════════════════════════════════════════════════╗
║           DASHBOARD CSV - PYTHON PURO (sin librerías)          ║
╚════════════════════════════════════════════════════════════════╝

📁 CARGAR DATOS:
   dashboard.cargar('clientes', 'ruta/clientes.csv')
   dashboard.cargar('facturas_enc', 'facturas_enc.csv')
   dashboard.cargar('ventas', 'ventas.csv')

👀 VER DATOS:
   dashboard.ver('clientes', 20)        # Ver 20 filas
   dashboard.listar()                    # Ver archivos cargados
   dashboard.resumen()                   # Resumen completo

📊 REPORTES:
   dashboard.ranking_clientes(10)        # Top 10 clientes
   dashboard.ticket_promedio(10)         # Ticket promedio
   dashboard.facturas_altas(10)          # Facturas más altas
   dashboard.ventas_por_mes()            # Ventas mensuales
   dashboard.producto_mas_vendido(5)     # Top 5 productos
   dashboard.ventas_por_rubro()          # Por rubro
   dashboard.top_productos_facturacion() # Top por $
   dashboard.top